In [99]:
import numpy as np
import tkinter as tk
from tkinter import messagebox, Entry, Button, Label
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import joblib
import tensorflow as tf
import vlc
import tkinter.font as font

In [100]:
tfidf = joblib.load('tfidf_vectorizer.pkl')
sentiment_model = joblib.load('sentiment_model.pkl')

In [101]:
user_disliked_video = False

In [102]:
def preprocess_text(text):
    return tfidf.transform([text])

In [103]:
instance = vlc.Instance()
player = instance.media_player_new()

videos = ['C:/Users/elyas/Desktop/New folder/Elyas.mp4']
current_video_index = 0  

In [104]:
def get_eeg_values_based_on_sentiment(sentiment_value):
    if sentiment_value == 1:  
        return {'Delta': [20, 30, 40], 'Theta': [10, 20, 15], 'Alpha': [40, 50, 60], 'Beta': [15, 20, 18], 'Gamma': [5, 7, 6]}
    elif sentiment_value == 0:  
        return {'Delta': [50, 60, 55], 'Theta': [30, 40, 35], 'Alpha': [30, 35, 32], 'Beta': [10, 15, 12], 'Gamma': [3, 4, 3.5]}
    else:  
        return {'Delta': [100, 120, 110], 'Theta': [60, 70, 65], 'Alpha': [20, 25, 22], 'Beta': [5, 8, 6], 'Gamma': [1, 2, 1.5]}


In [105]:
def plot_eeg_waves(eeg_data, sentiment):
    figure, ax = plt.subplots()
    colors = {'Delta': 'blue', 'Theta': 'orange', 'Alpha': 'green', 'Beta': 'red', 'Gamma': 'purple'}

    for band, values in eeg_data.items():
        ax.plot(values, label=band, color=colors[band], marker='o')  

    ax.set_title(f'Simulated EEG plot ({sentiment} Sentiment)')
    ax.set_xlabel('Time Points')
    ax.set_ylabel('EEG Amplitude (µV)') 
    ax.legend()
    ax.grid(True)

    ax.set_ylim([0, 200])  

    return figure

In [106]:
def set_comment_title(text):
    comment_title.config(text=text)

In [107]:
def play_next_video():
    global current_video_index
    if player.is_playing():
        player.stop()  
    current_video_index += 1  
    if current_video_index >= len(videos):
        current_video_index = 0  
    play_video(videos[current_video_index])

In [108]:
def play_video(video_path):
    media = instance.media_new(video_path)
    player.set_media(media)
    video_handle = video_label.winfo_id()
    player.set_hwnd(video_handle)
    player.play()

In [109]:
def like_video():
    set_comment_title("Please Leave a Comment")
    play_next_video()

In [110]:
def skip_video():
    global user_disliked_video
    set_comment_title("Please Leave a Comment")
    play_next_video()
    user_disliked_video = True

In [111]:
def simulate_negative_sentiment():
    negative_sentiment = -1  

   
    eeg_data = get_eeg_values_based_on_sentiment(negative_sentiment)

    for widget in plot_frame.winfo_children():
        widget.destroy()

    figure = plot_eeg_waves(eeg_data, 'Negative')
    canvas = FigureCanvasTkAgg(figure, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()

    result_label.config(text="Sentiment Analysis: Negative")

In [112]:
def stop_video():
    if player.is_playing():
        player.stop()
    stop_button.config(text="Finish")

In [113]:
def analyze_comment():
    global user_disliked_video
    user_comment = comment_entry.get()  
    try:
        if user_disliked_video:
            sentiment_prediction = -1
            user_disliked_video = False  
        else:
            preprocessed_text = preprocess_text(user_comment)  
            sentiment_prediction = sentiment_model.predict(preprocessed_text)[0]

        eeg_data = get_eeg_values_based_on_sentiment(sentiment_prediction)

        for widget in plot_frame.winfo_children():
            widget.destroy()

        if sentiment_prediction > 0:
            sentiment = 'Positive'
        elif sentiment_prediction == 0:
            sentiment = 'Neutral'
        else:
            sentiment = 'Negative'

        figure = plot_eeg_waves(eeg_data, sentiment)
        canvas = FigureCanvasTkAgg(figure, master=plot_frame)
        canvas.draw()
        canvas.get_tk_widget().pack()

        result_label.config(text=f"Sentiment Analysis: {sentiment}")
    except Exception as e:
        messagebox.showerror("Error", str(e))

In [114]:
root = tk.Tk()
root.title("User Interface")
result_label = Label(root, text="", font=("Helvetica", 14))
result_label.pack()

In [115]:
plot_frame = tk.Frame(root)
plot_frame.pack(fill=tk.BOTH, expand=True)

video_frame = tk.Frame(root, bg="black")
video_frame.pack(fill=tk.BOTH, expand=True)

video_label = tk.Label(video_frame, width=20, height=20)
video_label.pack(fill=tk.BOTH, expand=True)

comment_font = font.Font(size=50)  
comment_title = Label(root, text="", font=("Helvetica", 24), pady=70)
comment_title.pack()

comment_entry = Entry(root, font=comment_font)
comment_entry.pack()

analyze_button = Button(root, text="Analyze Comment", command=analyze_comment)
analyze_button.pack()

like_button = Button(video_frame, text="Like Video", command=like_video, bg="white", fg="black")
like_button.pack(side=tk.LEFT, padx=5, pady=5)

skip_button = Button(video_frame, text="Dislike Video", command=skip_video, bg="white", fg="black")
skip_button.pack(side=tk.LEFT, padx=5, pady=5)

stop_button = Button(video_frame, text="Stop Video", command=stop_video, bg="white", fg="black")
stop_button.pack(side=tk.LEFT, padx=5, pady=5)

In [116]:
play_video(videos[current_video_index])
root.mainloop()